In [1]:
import pandas as pd
import ast
import numpy as np
from pathlib import Path  
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import geopandas as gpd
import pandas as pd
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import seaborn as sns
import matplotlib.pyplot as plt
from adjustText import adjust_text
from matplotlib.patches import Patch
from sklearn.decomposition import PCA, FactorAnalysis
import matplotlib.colors as mcolors


# Load the Excel file
#df_transit = pd.read_excel('../../data/raw/2020-Fuel-and-Energy.xlsm', sheet_name='Fuel and Energy')

city_counties = pd.read_excel('../../data/raw/uscities.xlsx')


df_transit = pd.read_excel('../../data/raw/2020-Fuel-and-Energy.xlsm', sheet_name='Fuel and Energy')
df_transit.head(5)

C:\Users\peiyaozhao\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


,Agency,City,State,Legacy NTD ID,NTD ID,Organization Type,Primary UZA Population,Agency VOMS,Mode,TOS,...,Other Fuel (mpg) Questionable,Electric Propulsion (mi/kwh),Electric Propulsion (mi/kwh) Questionable,Electric Battery (mi/kwh),Electric Battery (mi/kwh) Questionable,Any data questionable?,Unnamed: 59,Unnamed: 60,1,Unnamed: 62
0,MTA New York City Transit,Brooklyn,NY,2008,20008,"Subsidiary Unit of a Transit Agency, Reporting...",18351295,10427,HR,DO,...,NaN,0.2102,NaN,NaN,NaN,No,NaN,NaN,Hide questionable data tags,NaN
1,MTA New York City Transit,Brooklyn,NY,2008,20008,"Subsidiary Unit of a Transit Agency, Reporting...",18351295,10427,CB,DO,...,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,Show questionable data tags,NaN
2,MTA New York City Transit,Brooklyn,NY,2008,20008,"Subsidiary Unit of a Transit Agency, Reporting...",18351295,10427,MB,DO,...,NaN,NaN,NaN,0.2483,NaN,No,NaN,NaN,1,1.0
3,MTA New York City Transit,Brooklyn,NY,2008,20008,"Subsidiary Unit of a Transit Agency, Reporting...",18351295,10427,RB,DO,...,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN
4,MTA New York City Transit,Brooklyn,NY,2008,20008,"Subsidiary Unit of a Transit Agency, Reporting...",18351295,10427,DR,PT,...,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN


In [2]:
# Identify columns with NaN values
cols_with_nan = df_transit.columns[df_transit.isna().any()].tolist()

# Ensure "City" column is not in the list of columns to drop
cols_to_drop = [col for col in cols_with_nan if col != 'City']

# Drop the identified columns
df_transit = df_transit.drop(columns=cols_to_drop)

df_transit

,Agency,City,State,NTD ID,Organization Type,Primary UZA Population,Agency VOMS,Mode,TOS,Mode VOMS,...,Electric Propulsion (kwh),Electric Battery (kwh),Diesel,Gasoline,Liquefied Petroleum Gas,Compressed Natural Gas,Other Fuel,Electric Propulsion,Electric Battery,Any data questionable?
0,MTA New York City Transit,Brooklyn,NY,20008,"Subsidiary Unit of a Transit Agency, Reporting...",18351295,10427,HR,DO,5413,...,1596235000,0,0,0,0,0,0,335497633,0,No
1,MTA New York City Transit,Brooklyn,NY,20008,"Subsidiary Unit of a Transit Agency, Reporting...",18351295,10427,CB,DO,435,...,0,0,14383040,0,0,0,0,0,0,No
2,MTA New York City Transit,Brooklyn,NY,20008,"Subsidiary Unit of a Transit Agency, Reporting...",18351295,10427,MB,DO,3201,...,0,449235,77895324,0,0,11633307,0,0,111558,No
3,MTA New York City Transit,Brooklyn,NY,20008,"Subsidiary Unit of a Transit Agency, Reporting...",18351295,10427,RB,DO,156,...,0,0,2792754,0,0,0,0,0,0,No
4,MTA New York City Transit,Brooklyn,NY,20008,"Subsidiary Unit of a Transit Agency, Reporting...",18351295,10427,DR,PT,1222,...,0,0,424795,24447143,0,0,0,0,0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1316,City of Milwaukee,Milwaukee,WI,55312,"City, County or Local Government Unit or Depar...",1376476,3,SR,PT,3,...,796799,0,0,0,0,0,0,72466,0,No
1317,Bay State Cruise Company,NaN,MA,11238,Private-For-Profit Corporation,246695,2,FB,DO,2,...,0,0,0,0,0,0,0,0,0,No
1318,"County of Pierce, dba: Pierce County Ferry",Tacoma,WA,28,"City, County or Local Government Unit or Depar...",3059393,2,FB,PT,2,...,0,0,42733,0,0,0,0,0,0,No
1319,Rhode Island Department of Transportation,Providence,RI,11147,State Government Unit or Department of Transpo...,1190956,1,FB,PT,1,...,0,0,19489,0,0,0,0,0,0,No


In [3]:
df_transit = df_transit.drop(df_transit.columns[-1], axis=1)
df_transit

,Agency,City,State,NTD ID,Organization Type,Primary UZA Population,Agency VOMS,Mode,TOS,Mode VOMS,...,Other Fuel (gal/gal equivalent),Electric Propulsion (kwh),Electric Battery (kwh),Diesel,Gasoline,Liquefied Petroleum Gas,Compressed Natural Gas,Other Fuel,Electric Propulsion,Electric Battery
0,MTA New York City Transit,Brooklyn,NY,20008,"Subsidiary Unit of a Transit Agency, Reporting...",18351295,10427,HR,DO,5413,...,0,1596235000,0,0,0,0,0,0,335497633,0
1,MTA New York City Transit,Brooklyn,NY,20008,"Subsidiary Unit of a Transit Agency, Reporting...",18351295,10427,CB,DO,435,...,0,0,0,14383040,0,0,0,0,0,0
2,MTA New York City Transit,Brooklyn,NY,20008,"Subsidiary Unit of a Transit Agency, Reporting...",18351295,10427,MB,DO,3201,...,0,0,449235,77895324,0,0,11633307,0,0,111558
3,MTA New York City Transit,Brooklyn,NY,20008,"Subsidiary Unit of a Transit Agency, Reporting...",18351295,10427,RB,DO,156,...,0,0,0,2792754,0,0,0,0,0,0
4,MTA New York City Transit,Brooklyn,NY,20008,"Subsidiary Unit of a Transit Agency, Reporting...",18351295,10427,DR,PT,1222,...,0,0,0,424795,24447143,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1316,City of Milwaukee,Milwaukee,WI,55312,"City, County or Local Government Unit or Depar...",1376476,3,SR,PT,3,...,0,796799,0,0,0,0,0,0,72466,0
1317,Bay State Cruise Company,NaN,MA,11238,Private-For-Profit Corporation,246695,2,FB,DO,2,...,0,0,0,0,0,0,0,0,0,0
1318,"County of Pierce, dba: Pierce County Ferry",Tacoma,WA,28,"City, County or Local Government Unit or Depar...",3059393,2,FB,PT,2,...,0,0,0,42733,0,0,0,0,0,0
1319,Rhode Island Department of Transportation,Providence,RI,11147,State Government Unit or Department of Transpo...,1190956,1,FB,PT,1,...,0,0,0,19489,0,0,0,0,0,0


In [4]:
df_transit.columns

Index(['Agency', 'City', 'State', 'NTD ID', 'Organization Type',
       'Primary UZA Population', 'Agency VOMS', 'Mode', 'TOS', 'Mode VOMS',
       'Diesel (gal)', 'Gasoline (gal)',
       'Liquefied Petroleum Gas (gal equivalent)',
       'Compressed Natural Gas (gal equivalent)', 'Bio-Diesel (gal)',
       'Other Fuel (gal/gal equivalent)', 'Electric Propulsion (kwh)',
       'Electric Battery (kwh)', 'Diesel', 'Gasoline',
       'Liquefied Petroleum Gas', 'Compressed Natural Gas', 'Other Fuel',
       'Electric Propulsion', 'Electric Battery'],
      dtype='object')

In [5]:
city_counties.head(5)

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,New York,New York,NY,New York,36081,Queens,40.6943,-73.9249,18908608,11080.3,shape,False,True,America/New_York,1,11229 11228 11226 11225 11224 11222 11221 1122...,1840034016
1,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1141,-118.4068,11922389,3184.7,shape,False,True,America/Los_Angeles,1,91367 90291 90293 90292 91316 91311 90035 9003...,1840020491
2,Chicago,Chicago,IL,Illinois,17031,Cook,41.8375,-87.6866,8497759,4614.5,shape,False,True,America/Chicago,1,60018 60649 60641 60640 60643 60642 60645 6064...,1840000494
3,Miami,Miami,FL,Florida,12086,Miami-Dade,25.7840,-80.2101,6080145,4758.9,shape,False,True,America/New_York,1,33128 33129 33125 33126 33127 33149 33144 3314...,1840015149
4,Houston,Houston,TX,Texas,48201,Harris,29.7860,-95.3885,5970127,1384.0,shape,False,True,America/Chicago,1,77069 77068 77061 77060 77063 77062 77065 7706...,1840020925


In [9]:
# Rename 'city' to 'City' and 'state_id' to 'State' for consistency in city_counties
city_counties.rename(columns={'city': 'City', 'state_id': 'State'}, inplace=True)

# Create a set of unique (City, State) pairs from df_transit
transit_cities_states = set(zip(df_transit['City'], df_transit['State']))

# Create a set of unique (City, State) pairs from city_counties
cities_states = set(zip(city_counties['City'], city_counties['State']))

# Check for matches
matches = transit_cities_states.intersection(cities_states)

# Check if all cities in df_transit are covered by the matches
all_covered = transit_cities_states.issubset(matches)

# Print result
if all_covered:
    print("All cities in df_transit are covered by the matched cities in df_cities.")
else:
    print("Some cities in df_transit are not covered by the matched cities in df_cities.")
    # Print the cities not covered
    uncovered_cities = transit_cities_states - matches
    print("Cities not covered:", uncovered_cities)

# Select relevant columns from city_counties for merging
city_counties_subset = city_counties[['City', 'State', 'county_name']]

# Merge df_transit with city_counties_subset on 'City' and 'State'
merged_df = pd.merge(df_transit, city_counties_subset, on=['City', 'State'], how='left')

# # Extract specific columns from merged_df
# transit_counties_df = merged_df[['Agency', 'City', 'State', 'county_name', 'Diesel (gal)', 'Gasoline (gal)','Diesel', 'Gasoline']]

# # Rename 'county_name' to 'County' for clarity
# transit_counties_df.rename(columns={'county_name': 'County','Diesel': 'Diesel (miles)','Gasoline': 'Gasoline (miles)'}, inplace=True)

# transit_counties_df
transit_counties_df = merged_df[['Agency', 'City', 'State', 'county_name', 'Diesel (gal)', 'Gasoline (gal)',
       'Liquefied Petroleum Gas (gal equivalent)',
       'Compressed Natural Gas (gal equivalent)', 'Bio-Diesel (gal)',
       'Other Fuel (gal/gal equivalent)', 'Electric Propulsion (kwh)',
       'Electric Battery (kwh)', 'Diesel', 'Gasoline',
       'Liquefied Petroleum Gas', 'Compressed Natural Gas', 'Other Fuel',
       'Electric Propulsion', 'Electric Battery']]
transit_counties_df

Some cities in df_transit are not covered by the matched cities in df_cities.
Cities not covered: {('Madison', 'TN'), ('New York', 'NJ'), ('Jamaica', 'NY'), ('Saint Petersburg', 'FL'), ('Carmel', 'NY'), ('Tucson   ', 'AZ'), ('Weehawken', 'NJ'), ('Hamden', 'CT'), ('Mahwah', 'NJ'), ('Hyannis', 'MA'), ('Preston', 'CT'), (nan, 'PR'), ('Berlin', 'CT'), ('Watertown', 'CT'), ('Newington', 'CT'), ('Saint Louis', 'MO'), ('Winston Salem', 'NC'), (nan, 'NJ'), ('Swiftwater', 'PA'), ('Huntington', 'NY'), ('Hurley', 'NY'), ('Montclair', 'NJ'), ('Dekalb', 'IL'), (nan, 'MA'), ('Saint Cloud', 'MN'), ('Milford', 'CT')}


,Agency,City,State,county_name,Diesel (gal),Gasoline (gal),Liquefied Petroleum Gas (gal equivalent),Compressed Natural Gas (gal equivalent),Bio-Diesel (gal),Other Fuel (gal/gal equivalent),Electric Propulsion (kwh),Electric Battery (kwh),Diesel,Gasoline,Liquefied Petroleum Gas,Compressed Natural Gas,Other Fuel,Electric Propulsion,Electric Battery
0,MTA New York City Transit,Brooklyn,NY,Kings,0,0,0,0,0,0,1596235000,0,0,0,0,0,0,335497633,0
1,MTA New York City Transit,Brooklyn,NY,Kings,3290382,0,0,0,0,0,0,0,14383040,0,0,0,0,0,0
2,MTA New York City Transit,Brooklyn,NY,Kings,23223866,0,0,3857115,0,0,0,449235,77895324,0,0,11633307,0,0,111558
3,MTA New York City Transit,Brooklyn,NY,Kings,1074143,0,0,0,0,0,0,0,2792754,0,0,0,0,0,0
4,MTA New York City Transit,Brooklyn,NY,Kings,146008,3522121,0,0,0,0,0,0,424795,24447143,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1316,City of Milwaukee,Milwaukee,WI,Milwaukee,0,0,0,0,0,0,796799,0,0,0,0,0,0,72466,0
1317,Bay State Cruise Company,NaN,MA,NaN,155873,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1318,"County of Pierce, dba: Pierce County Ferry",Tacoma,WA,Pierce,184209,0,0,0,0,0,0,0,42733,0,0,0,0,0,0
1319,Rhode Island Department of Transportation,Providence,RI,Providence,121806,0,0,0,0,0,0,0,19489,0,0,0,0,0,0


In [10]:
transit_counties_df.rename(columns={'county_name': 'County','Diesel':'Diesel (miles)', 'Gasoline':'Gasoline (miles)',
       'Liquefied Petroleum Gas':'Liquefied Petroleum Gas (miles)', 'Compressed Natural Gas':'Compressed Natural Gas (miles)', 'Other Fuel':'Other Fuel (miles)',
       'Electric Propulsion':'Electric Propulsion (miles)', 'Electric Battery':'Electric Battery (miles)'}, inplace=True)

transit_counties_df

C:\Users\peiyaozhao\AppData\Local\Temp\ipykernel_15964\2332995431.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transit_counties_df.rename(columns={'county_name': 'County','Diesel':'Diesel (miles)', 'Gasoline':'Gasoline (miles)',


,Agency,City,State,County,Diesel (gal),Gasoline (gal),Liquefied Petroleum Gas (gal equivalent),Compressed Natural Gas (gal equivalent),Bio-Diesel (gal),Other Fuel (gal/gal equivalent),Electric Propulsion (kwh),Electric Battery (kwh),Diesel (miles),Gasoline (miles),Liquefied Petroleum Gas (miles),Compressed Natural Gas (miles),Other Fuel (miles),Electric Propulsion (miles),Electric Battery (miles)
0,MTA New York City Transit,Brooklyn,NY,Kings,0,0,0,0,0,0,1596235000,0,0,0,0,0,0,335497633,0
1,MTA New York City Transit,Brooklyn,NY,Kings,3290382,0,0,0,0,0,0,0,14383040,0,0,0,0,0,0
2,MTA New York City Transit,Brooklyn,NY,Kings,23223866,0,0,3857115,0,0,0,449235,77895324,0,0,11633307,0,0,111558
3,MTA New York City Transit,Brooklyn,NY,Kings,1074143,0,0,0,0,0,0,0,2792754,0,0,0,0,0,0
4,MTA New York City Transit,Brooklyn,NY,Kings,146008,3522121,0,0,0,0,0,0,424795,24447143,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1316,City of Milwaukee,Milwaukee,WI,Milwaukee,0,0,0,0,0,0,796799,0,0,0,0,0,0,72466,0
1317,Bay State Cruise Company,NaN,MA,NaN,155873,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1318,"County of Pierce, dba: Pierce County Ferry",Tacoma,WA,Pierce,184209,0,0,0,0,0,0,0,42733,0,0,0,0,0,0
1319,Rhode Island Department of Transportation,Providence,RI,Providence,121806,0,0,0,0,0,0,0,19489,0,0,0,0,0,0


In [12]:
transit_counties_df.to_csv('../../data/tidy/transit-counties-vmt.csv', index=False)